In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Gweru,-19.45,29.82,66.72,45,0,11.63,ZW,2020-10-16 19:38:40
1,1,Micheweni,-4.97,39.83,77.92,81,30,11.54,TZ,2020-10-16 19:38:40
2,2,Kahului,20.89,-156.47,84.20,58,20,5.82,US,2020-10-16 19:36:04
3,3,Rikitea,-23.12,-134.97,73.74,81,100,21.81,PF,2020-10-16 19:35:07
4,4,Narodychi,51.20,29.08,49.89,78,99,4.34,UA,2020-10-16 19:38:40


In [4]:
city_data_df.dtypes

City_ID         int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date           object
dtype: object

In [5]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [7]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [11]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [15]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [16]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,Micheweni,-4.97,39.83,77.92,81,30,11.54,TZ,2020-10-16 19:38:40
2,2,Kahului,20.89,-156.47,84.20,58,20,5.82,US,2020-10-16 19:36:04
14,14,Sokoni,-6.43,39.55,78.80,83,40,11.41,TZ,2020-10-16 19:38:42
19,19,Vaini,-21.20,-175.20,75.20,94,90,10.29,TO,2020-10-16 19:38:42
24,24,Avera,33.19,-82.53,78.80,54,40,10.29,US,2020-10-16 19:38:43
26,26,Butaritari,3.07,172.79,81.37,76,0,14.27,KI,2020-10-16 19:38:43
28,28,Tautira,-17.73,-149.15,77.00,88,90,11.41,PF,2020-10-16 19:38:44
31,31,Hilo,19.73,-155.09,80.60,69,40,5.82,US,2020-10-16 19:36:28
32,32,Puerto Ayora,-0.74,-90.35,78.80,65,20,16.11,EC,2020-10-16 19:38:44
33,33,Pontian Kecil,1.48,103.39,80.01,83,20,5.30,MY,2020-10-16 19:38:44


In [17]:
preferred_cities_df.count()

City_ID       175
City          175
Lat           175
Lng           175
Max Temp      175
Humidity      175
Cloudiness    175
Wind Speed    175
Country       175
Date          175
dtype: int64

In [18]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Micheweni,TZ,77.92,-4.97,39.83,
2,Kahului,US,84.20,20.89,-156.47,
14,Sokoni,TZ,78.80,-6.43,39.55,
19,Vaini,TO,75.20,-21.20,-175.20,
24,Avera,US,78.80,33.19,-82.53,
26,Butaritari,KI,81.37,3.07,172.79,
28,Tautira,PF,77.00,-17.73,-149.15,
31,Hilo,US,80.60,19.73,-155.09,
32,Puerto Ayora,EC,78.80,-0.74,-90.35,
33,Pontian Kecil,MY,80.01,1.48,103.39,


In [19]:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"}
# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()

In [25]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [26]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Micheweni,TZ,77.92,-4.97,39.83,
2,Kahului,US,84.20,20.89,-156.47,Maui Seaside Hotel
14,Sokoni,TZ,78.80,-6.43,39.55,Ujamaa Beach Resort
19,Vaini,TO,75.20,-21.20,-175.20,Keleti Beach Resort
24,Avera,US,78.80,33.19,-82.53,
26,Butaritari,KI,81.37,3.07,172.79,Isles Sunset Lodge
28,Tautira,PF,77.00,-17.73,-149.15,Pension Oaoa
31,Hilo,US,80.60,19.73,-155.09,Hilo Hawaiian Hotel
32,Puerto Ayora,EC,78.80,-0.74,-90.35,Finch Bay Galapagos Hotel
33,Pontian Kecil,MY,80.01,1.48,103.39,SUNFLOWER EXPRESS HOTEL


In [27]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [28]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [29]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [30]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [31]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))